# Dynamic Programming

Dynamic Programming refers to a collection of algorithms that can be used to compute optimal policies given a perfect model of the environment as a Markov decision process. While classical DP is of limited use for reinforcement learning nowadays, because of their assumption of a perfect model and great computational expense, they are still the foundation for the understanding of more complex models and are important theoretically.

The key idea of DP is the use of value functions to organize and structure the search for good policies. 

### Grid World

We will use grid world toy example, where each agent can move up, down, left or right within a grid with previously specified rows and columns. Each move results in reward of -1. Top-left and bottom-right corners are terminal states are getting there gives agents the reward of 0. If agent makes a move that is out of bounds of the given grid, the agents stays in place instead and receives reward of -1.

In [270]:
import numpy as np

class GridWorld:
   def __init__(self, rows=4, cols=4):
      self.rows = rows
      self.cols = cols
      self.state = None
      self.is_done = False
      self.available_actions = ['up', 'down', 'left', 'right']
      self.a2i = {'up': 0, 'down': 1, 'left': 2, 'right': 3}
      self.i2a = {0: 'up', 1: 'down', 2: 'left', 3: 'right'}
      self.reset()
         
   def reset(self):
      self.grid = np.zeros((self.rows, self.cols))
      for i in range(self.rows):
         self.grid[i, :] = np.arange(self.cols)+self.cols*i
      
      self.state = np.random.randint(1, self.rows*self.cols)
      self.n_states = self.rows*self.cols
      self.is_done = False
      
   def step(self, action, state=None, look_ahead=False):
      if self.is_done:
         print('Agent reached terminal state in the previous move, please reset the environment!')
         return
      if state is None: state = self.state
      
      action = action.lower()
      if action not in self.available_actions:
         raise 'Invalid action'
      
      terminal = False
      reward = -1
      if action == 'up':
         new_state = state - self.rows if state not in self.grid[0, :] else state
      elif action == 'down':
         new_state = state + self.rows if state not in self.grid[self.rows-1, :] else state
      elif action == 'right':
         new_state = state + 1 if state not in self.grid[:, self.cols-1] else state
      elif action == 'left':
         new_state = state - 1 if state not in self.grid[:, 0] else state
         
      if self.is_terminal(new_state):
         terminal = True
         reward = -1

      if not look_ahead:
         self.state = new_state
         self.is_done = terminal
         
      return new_state, reward, terminal
   
   def is_terminal(self, state):
      return state == 0 or state == (self.n_states-1)
   
   def __repr__(self):
      vis = np.zeros_like(self.grid)
      row = self.state // self.rows
      col = self.state % self.cols
      vis[row][col] = 1
      return str(vis)

In [271]:
g = GridWorld()
g

[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

In [272]:
g.step('down')

(9, -1, False)

In [273]:
g

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]]

### Policy Evaluation

First, let's look into ways of computing the state-value function $v_\pi$ for an arbitrary policy $\pi$. We know that:

$$v_\pi(s)=E_\pi [G_t|S_t=s]=\sum_a \pi(a|s)\sum_{s',r}p(s',r|s,a)[r+\gamma v_\pi (s')]$$

where $\pi(a|s)$ is the probability of taking action $a$ in state $s$ under policy $\pi$. The expectations $E_\pi$ are conditional on $\pi$, thus the subscript.

We can use *iterative policy evaluation* which applies the same operation to each state $s$: it replaces the old value of $s$ with new value obtained form the old values of the successor states of $s$, and the expected immediate rewards, along all the one-step transitions possible under the policy being evaluated. This is called *expected update* - each iteration of iterative policy evalution updates the value of every state once to produce the new approximate value function $v_{k+1}$. The updates are called *expected*, because they are based on expectation over all possible next states rather than on a sample next state. 

<img src='resources/iterative-policy-evaluation.JPG' />

In [343]:
import numpy as np

def ipe(env, policy, theta=1e-5, gamma=1, n_steps=100, print_every=False):
   '''
   Iterative Policy Evaluation
   Inputs:
    - pi (dict): policy to be evaluated
    - theta (float): threshold > 0 determining accuracy of estimation
    - gamma (float): discount factor how to measure expected rewards received in far future
    - n_steps (int): for how many steps to iterate through all states
    - print_every (list): whether to print the curret value table estimation
   '''
   V = np.zeros(env.n_states)
   i_step = 0.
   while True and i_step < n_steps:
      delta = 0.
      V_tmp = V.copy()
      
      # Calculate
      for s in range(1, env.n_states-1):
         v = 0.
         for a in env.available_actions:
            new_s, r, done = env.step(a, state=s, look_ahead=True)
            v += policy[s][a] * (r + gamma*V[new_s])
         delta = max(delta, np.abs(v-V_tmp[s]))
         V_tmp[s] = v
         #if delta < theta: break
      
      # Print value estimation table
      if print_every:
         if i_step in print_every:
            print(f'Iteration: {i_step}')
            print(np.round(V.reshape(env.rows, env.cols), 1))
            print('-'*50 + '\n')
            
      V = V_tmp.copy()
      i_step += 1
      
   return V

In [316]:
random_policy = {key: {val: 0.25 for val in ['up', 'down', 'left', 'right']}for key in range(0, 16)}

In [317]:
g.reset()
V = ipe(g, random_policy, n_steps=100, print_every=[0, 1, 2, 3, 10, 99])

Iteration: 0.0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
--------------------------------------------------

Iteration: 1.0
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
--------------------------------------------------

Iteration: 2.0
[[ 0.  -1.8 -2.  -2. ]
 [-1.8 -2.  -2.  -2. ]
 [-2.  -2.  -2.  -1.8]
 [-2.  -2.  -1.8  0. ]]
--------------------------------------------------

Iteration: 3.0
[[ 0.  -2.4 -2.9 -3. ]
 [-2.4 -2.9 -3.  -2.9]
 [-2.9 -3.  -2.9 -2.4]
 [-3.  -2.9 -2.4  0. ]]
--------------------------------------------------

Iteration: 10.0
[[ 0.  -6.1 -8.4 -9. ]
 [-6.1 -7.7 -8.4 -8.4]
 [-8.4 -8.4 -7.7 -6.1]
 [-9.  -8.4 -6.1  0. ]]
--------------------------------------------------

Iteration: 99.0
[[  0.  -13.9 -19.9 -21.9]
 [-13.9 -17.9 -19.9 -19.9]
 [-19.9 -19.9 -17.9 -13.9]
 [-21.9 -19.9 -13.9   0. ]]
--------------------------------------------------



### Policy Improvement

One of the reasons for computing the value function is to find better policies. We know how good it is to follow the current policy from which the value function was estimated in the first place, but is it better or worse to change to the new policy? One way to answer that is to select $a$ in $s$ and follow the existing policy:

$$q_\pi(s,a)=\sum_{s',r}p(s',r|s,a)[r+\gamma v_\pi (s')]$$

The criteria whether new action is better is dependent whether it is greater than or less than $v_\pi(s)$. This check is called *policy improvement theorem*:

$$q_\pi(s,\pi'(s)) \geq v_\pi(s)$$

If the above equation is true, then $\pi'$ must be as good as, or better than, $\pi$ - it must obtain greater or equal expected return from all states $s \in S$

$$v_{\pi'}(s) \geq v_\pi(s)$$

Thus, knowing a policy and its value function, we can easily evaluate a change in he policy at a single state to a particular action. We can extend this to consider changes at *all* states and to *all* possible actions, selecting at each state the action that appears best according to $q_\pi(s)$

$$\pi'(s)=argmax_a q_\pi(s,a)=argmax_a \sum_{s',r}p(s',r|s,a)[r+\gamma v_\pi(s')]$$

This *greedy* policy takes action that looks best in short term - after one step of lookahead - according to $v$. By construction, if meets the conditions of the policy improvement theorem and thus must be as good (or better) than the original policy.

We can iterate through policies by improving the first policy $\pi$ using $v_\pi$, which yields $\pi'$. Then we can compute $v_{\pi'}$ and improve it again to yield even better $\pi''$ until we reach convergance at optimal $v_*$ and $\pi_*$.

<img src='./resources/policy-iteration.JPG' />

In [426]:
def ipi(env, policy, V, gamma=1.):
   '''Iterative Policy Improvement'''
   policy_stable = True
   for s in range(1, env.n_states-1):
      old_action = np.argmax(list(policy[s].values()))
      
      action_eval = []
      for a in env.available_actions:
         new_s, r, done = env.step(a, state=s, look_ahead=True)
         action_eval.append(r + gamma*V[new_s])
      best_action = np.argmax(action_eval)
      
      if old_action != best_action:
         policy_stable = False
      
      for a in env.available_actions:
         if env.a2i[a] == best_action: 
            policy[s][a] = 1.
         else: 
            policy[s][a] = 0.

   return policy, policy_stable

In [435]:
def policy_iteration(env):
   # Start with random policy
   policy = {key: {val: 1./len(env.available_actions) for val in env.available_actions} for key in range(0, env.n_states)}
   done = False
   
   while not done:
      # Policy Evaluation
      V = ipe(env, policy, theta=1e-5, gamma=1, n_steps=100, print_every=False)

      # Policy Improvement
      policy, done = ipi(env, policy, V)

   return policy, V

In [437]:
g.reset()
policy, V = policy_iteration(g)

In [438]:
policy

{0: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25},
 1: {'up': 0.0, 'down': 0.0, 'left': 1.0, 'right': 0.0},
 2: {'up': 0.0, 'down': 0.0, 'left': 1.0, 'right': 0.0},
 3: {'up': 0.0, 'down': 1.0, 'left': 0.0, 'right': 0.0},
 4: {'up': 1.0, 'down': 0.0, 'left': 0.0, 'right': 0.0},
 5: {'up': 1.0, 'down': 0.0, 'left': 0.0, 'right': 0.0},
 6: {'up': 1.0, 'down': 0.0, 'left': 0.0, 'right': 0.0},
 7: {'up': 0.0, 'down': 1.0, 'left': 0.0, 'right': 0.0},
 8: {'up': 1.0, 'down': 0.0, 'left': 0.0, 'right': 0.0},
 9: {'up': 1.0, 'down': 0.0, 'left': 0.0, 'right': 0.0},
 10: {'up': 0.0, 'down': 1.0, 'left': 0.0, 'right': 0.0},
 11: {'up': 0.0, 'down': 1.0, 'left': 0.0, 'right': 0.0},
 12: {'up': 1.0, 'down': 0.0, 'left': 0.0, 'right': 0.0},
 13: {'up': 0.0, 'down': 0.0, 'left': 0.0, 'right': 1.0},
 14: {'up': 0.0, 'down': 0.0, 'left': 0.0, 'right': 1.0},
 15: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}}

In [439]:
V.reshape(g.rows, g.cols)

array([[ 0., -1., -2., -3.],
       [-1., -2., -3., -2.],
       [-2., -3., -2., -1.],
       [-3., -2., -1.,  0.]])

In [463]:
pick_best_action = lambda env, policy, state: env.i2a[np.argmax(list(policy[state].values()))]
optimal_policy = np.array([pick_best_action(env, policy, s) for s in range(env.n_states)]).reshape(env.rows, env.cols)
print(optimal_policy)

[['up' 'left' 'left' 'down']
 ['up' 'up' 'up' 'down']
 ['up' 'up' 'down' 'down']
 ['up' 'right' 'right' 'up']]
